In [1]:
import pandas as pd
import numpy as np

In [2]:
def gini(y):
    p = y.value_counts()/y.shape[0]
    gini = 1-np.sum(p**2)
    return gini
def entropy(y):
    a = y.value_counts()/y.shape[0]
    entropy = np.sum(-a*np.log2(a+1e-9))
    return entropy 

def split(train,criterion, min_instances):
    
    best_gain = -np.inf
    best_feature = None
    best_threshold= None
    
    for feature in train.columns[:-1]:
        for threshold in train[feature].unique():
            left = train[train[feature] < threshold]
            right = train[train[feature] >= threshold]
            if (len(left) < min_instances or len(right) < min_instances):
                break
            
            gain = criterion(train['track_genre']) - (len(left)/len(train) * criterion(left['track_genre']) + len(right)/len(train) * criterion(right['track_genre']))
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = threshold
    return (best_feature,best_threshold)


In [3]:
train_df = pd.read_csv('spotify_train.csv')
test_df = pd.read_csv('spotify_test.csv')
# left = df[df['key'] > 1]
# left.head()
train_df.head()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,211682,True,0.732,0.635,1,-7.891,1,0.4100,0.4930,0.000007,0.3880,0.581,147.025,4,hip-hop
1,1,223613,False,0.409,0.570,6,-10.540,0,0.0711,0.6870,0.000000,0.1730,0.336,128.657,4,pop
2,56,243626,False,0.536,0.764,1,-5.174,0,0.0393,0.0302,0.000011,0.1040,0.294,147.585,4,rock
3,8,182413,True,0.843,0.789,7,-2.801,1,0.2470,0.0028,0.000024,0.0322,0.571,125.071,4,hip-hop
4,74,200120,False,0.535,0.765,1,-7.862,0,0.0444,0.0540,0.000000,0.0921,0.373,191.827,4,pop


In [24]:
def dtree(train, criterion, max_depth=None, min_instances=2, target_impurity=0.0):

    majority_class = train['track_genre'].mode()[0]
   
    examples_in_split = len(train['track_genre'])
    
    
    classes = train['track_genre']
    impurity_score = criterion(classes)
    if (max_depth is not None and max_depth <= 1):  ##If the max_depth is defined and has been reached return a leaf node
        return (None,None, examples_in_split, majority_class,impurity_score,0,None,None)
    
    if (examples_in_split < min_instances):  ##If not enough instances then stop splitting
        return (None,None, examples_in_split, majority_class,impurity_score,0,None,None)
    
    
    if (impurity_score <= target_impurity):   ## Halt splitting if impurity 
        return (None,None, examples_in_split, majority_class,impurity_score,0,None,None)
    
    
    feature, threshold = split(train,criterion,min_instances)
    
    if (feature is None or threshold is None):
         return (None,None, examples_in_split, majority_class,impurity_score,0,None,None)
    
    train_left = train[train[feature] < threshold]
    train_right = train[train[feature] >= threshold]
    
    left_sub = dtree(train_left, criterion, max_depth - 1 if max_depth is not None else None, min_instances, target_impurity)
    
    right_sub = dtree(train_right, criterion, max_depth - 1 if max_depth is not None else None, min_instances, target_impurity)
    
    return (feature, threshold, examples_in_split, majority_class, impurity_score, f'depth: {abs(max_depth-12)}', left_sub, right_sub)

In [5]:
def predict(model,data):
    preds = []
    actual = data['track_genre'].tolist()
    for i, row in data.iterrows():
        node = model
        while node[6] is not None and node[7] is not None:
            field,threshold,_,_,_,_,left,right = node
            if row[field] < threshold:
                node = left
            else:
                node = right
        preds.append(node[3])
    #accuracy = sum(1 for x,y in zip(preds,actual) if x == y) / len(actual)
    return pd.Series(preds) # accuracy
    

In [6]:
def k_fold_cv(data,k=10):
    accuracy_scores = []
    fold_size = len(data) // k
    
    for i in range(k):
        test_start = i * fold_size
        test_end = (i+1) * fold_size
        test_indices = range(test_start,test_end)
        train_indices = list(set(range(len(data))) - set(test_indices))
        
        train_data = data.iloc[train_indices]
        test_data = data.iloc[test_indices]
        
        current_model = dtree(train_data, gini, max_depth=12, min_instances=2, target_impurity=0.22)
        
        predictions = predict(current_model,test_data)
        
        actual = test_data['track_genre']
        actual.name = None
        actual.index = range(fold_size)
        #print(type(predictions))
       # print(type(actual))
        accuracy = sum(1 for x,y in zip(predictions,actual) if x == y) / len(actual)#np.mean(predictions == actual)
        
        accuracy_scores.append(accuracy)
        #print(accuracy)
        
    return np.mean(accuracy_scores)

Best model so far: dtree(train_data, gini, max_depth=12, min_instances=2, target_impurity=0.22) -> 0.72899

In [7]:
k_fold_cv(train_df,10)

0.7289999999999999

In [25]:
model = dtree(train_df, gini, max_depth=12, min_instances=2, target_impurity=0.22)

In [21]:
predictions = predict(model, test_df)
actual = test_df['track_genre'].tolist()
accuracy = sum(1 for x,y in zip(predictions,actual) if x == y) / len(actual)

In [22]:
accuracy

0.732

In [26]:
model

('danceability',
 0.703,
 2000,
 'rock',
 0.6666415,
 'depth: 0',
 ('popularity',
  56,
  1256,
  'rock',
  0.619780467767455,
  'depth: 1',
  ('danceability',
   0.623,
   679,
   'rock',
   0.46773714268362254,
   'depth: 2',
   ('energy',
    0.265,
    486,
    'rock',
    0.3659926501718911,
    'depth: 3',
    ('loudness',
     -11.417,
     25,
     'pop',
     0.31999999999999984,
     'depth: 4',
     ('valence',
      0.204,
      8,
      'rock',
      0.46875,
      'depth: 5',
      (None, None, 2, 'pop', 0.0, 0, None, None),
      ('duration_ms',
       204466,
       6,
       'rock',
       0.2777777777777777,
       'depth: 6',
       (None, None, 2, 'pop', 0.5, 0, None, None),
       (None, None, 4, 'rock', 0.0, 0, None, None))),
     (None, None, 17, 'pop', 0.0, 0, None, None)),
    ('speechiness',
     0.218,
     461,
     'rock',
     0.3287016341914447,
     'depth: 4',
     ('valence',
      0.475,
      449,
      'rock',
      0.30150644094027323,
      'depth

In [11]:
# CAN IGNORE BELOW, JUST TESTING

In [12]:
#testing different models
#model1 = dtree(train_df, gini, max_depth=10, min_instances=2, target_impurity=0.0)
#predict(model1, test_df)

In [13]:
#model2 = dtree(train_df, gini, max_depth=8, min_instances=2, target_impurity=0.15)
#predict(model2, test_df)

In [14]:
#model3 = dtree(train_df, gini, max_depth=8, min_instances=2, target_impurity=0.19)
#predict(model3, test_df)